In [34]:
import torch
from torch import nn
from d2l import torch as d2l

In [31]:
class AlexNet(nn.Module):
    def __init__(self,**kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.con1 = nn.Conv2d(1, 96, kernel_size=(11,11), stride=4)
        self.relu1 = nn.ReLU()
        self.maxPool1 = nn.MaxPool2d(kernel_size=(3,3), stride=2)
        self.con2 = nn.Conv2d(96, 256, kernel_size=(5,5), padding=2)
        self.relu2 = nn.ReLU()
        self.maxPool2 = nn.MaxPool2d(kernel_size=(3,3), stride=2)
        self.con3 = nn.Conv2d(256, 384, kernel_size=(3,3), padding=1)
        self.relu3 = nn.ReLU()
        self.con4 = nn.Conv2d(384, 384, kernel_size=(3,3), padding=1)
        self.relu4 = nn.ReLU()
        self.con5 = nn.Conv2d(384, 256, kernel_size=(3,3), padding=1)
        self.relu5 = nn.ReLU()
        self.maxPool3 = nn.MaxPool2d(kernel_size=(3,3), stride=2)
        self.flat = nn.Flatten()
        self.ffn1 = nn.Linear(6400, 4096)
        self.relu6 = nn.ReLU()
        self.drop1 = nn.Dropout(0.5)
        self.ffn2 = nn.Linear(4096, 4096)
        self.relu7 = nn.ReLU()
        self.drop2 = nn.Dropout(0.5)
        self.ffn3 = nn.Linear(4096, 10)
        
    
    def forward(self, X):
        X = self.maxPool1(self.relu1(self.con1(X)))
        X = self.maxPool2(self.relu2(self.con2(X)))
        X = self.relu3(self.con3(X))
        X = self.relu4(self.con4(X))
        X = self.maxPool3(self.relu5(self.con5(X)))
        X = self.flat(X)
        X = self.relu6(self.ffn1(X))
        X = self.drop1(X)
        X = self.relu7(self.ffn2(X))
        X = self.drop2(X)
        Y = self.ffn3(X)

        return Y

net = AlexNet()


In [32]:
X = torch.ones((1,1, 224, 224))
# print(X.shape)
Y = net(X)
print(Y.shape)
# for layer in net:
#     X = layer(X)
#     print(f'layer:{net.__class__.__name__()}, output shape:{X.shape}')


torch.Size([1, 10])


In [35]:
batch_size = 128
lr, num_epochs = 0.01, 10
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)


In [ ]:
def train(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)

    print('train on:', device)
    net.to(device)

    optimizer = torch.optim.SGD(net.paremeters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    # 以下两行不用深究，拿过来用就行了。
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], legend=['train_loss', 'train_acc', 'test_acc'])